In [ ]:
!pip install transformers

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

In [ ]:
# 허깅페이스 transformers 에 등록된 사전 학습된 koGTP2 토크나이저를 가져온다.
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token = '</s>', eos_token = '</s>', unk_token = '<unk>',
  pad_token = '<pad>', mask_token = '<mask>')
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

# tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='', eos_token='', pad_token='')
# model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print('-' * 10)
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

1
1
3
----------
</s>
<usr>
<pad>
<sys>


In [ ]:
import pandas as pd
import tqdm
import urllib.request

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
                           filename = "ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [ ]:
len(train_data)

11823

In [ ]:
def get_chat_data():
  for questions, answers in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('' + questions + '' + answers)
    yield bos_token + sent + eos_token

In [ ]:
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types = tf.int32)

In [ ]:
batch_size = 32
dataset = dataset.padded_batch(batch_size = batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

ValueError: ignored

In [ ]:
for batch in dataset:
    print(batch)
    break

tf.Tensor(
[[    1  9349  7888   739  7318   376 25000  6824  9108  9028  7098 25856
      1     3     3     3     3     3     3     3     3     3     3     3
      3     3     3     3]
 [    1  9020  8263  7497 10192 11615  8210  8006 11567  8711  9535  7483
  12521     1     3     3     3     3     3     3     3     3     3     3
      3     3     3     3]
 [    1  9085  7597   395  8149 10624  7397 24224 13358  7182  8030 19138
  16899  9677  8234   389     1     3     3     3     3     3     3     3
      3     3     3     3]
 [    1  9085  7597   395  8149  9465 10624  7397 24224 13358  7182  8030
  19138 16899  9677  8234   389     1     3     3     3     3     3     3
      3     3     3     3]
 [    1  9943   422   418  9327  8702  7098  7141 16912 18328  8671  7415
   8263  8234   389     1     3     3     3     3     3     3     3     3
      3     3     3     3]
 [    1  9815   410 21249 10174  6824  8210  8006 16146 11056 11594 10137
  10556  9266  8711 25856     1     3   

In [ ]:
tokenizer.decode(batch[0])

'</s> 12시 땡!하루가 또 가네요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [ ]:
print(batch[0])

tf.Tensor(
[    1  9349  7888   739  7318   376 25000  6824  9108  9028  7098 25856
     1     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3], shape=(28,), dtype=int32)


In [ ]:
print(tokenizer.encode(' 12시 땡! 하루가 또 가네요.'))

[9349, 7888, 739, 7318, 376, 12557, 6824, 9108, 9028, 7098, 25856]


In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate = 3e-5, epsilon = 1e-08)

In [ ]:
steps = len(train_data) // batch_size + 1
print(steps)

370


In [ ]:
EPOCHS = 3

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.tqdm_notebook(dataset, total = steps):
      with tf.GradientTape() as tape:
          result = model(batch, labels = batch)
          loss = result[0]
          batch_loss = tf.reduce_mean(loss)

      grads = tape.gradient(batch_loss, model.trainable_variables)
      adam.apply_gradients(zip(grads, model.trainable_variables))
      epoch_loss += batch_loss / steps

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

<ipython-input-51-32d20f966673>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm.tqdm_notebook(dataset, total = steps):


  0%|          | 0/370 [00:00<?, ?it/s]

TypeError: ignored

In [ ]:
text = '오늘도 좋은 하루!'

In [ ]:
sent = '' + text + ''

In [ ]:
input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])

In [ ]:
output = model.generate(input_ids, max_length=50, early_stopping=True, eos_token_id=tokenizer.eos_token_id)

In [ ]:
decoded_sentence = tokenizer.decode(output[0].numpy().tolist())

In [ ]:
decoded_sentence.split(' ')[1].replace('', '')

In [ ]:
output = model.generate(input_ids, max_length=50, do_sample=True, top_k=10)
tokenizer.decode(output[0].numpy().tolist())

In [ ]:
def return_answer_by_chatbot(user_text):
  sent = '' + user_text + ''
  input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
  input_ids = tf.convert_to_tensor([input_ids])
  output = model.generate(input_ids, max_length=50, do_sample=True, top_k=20)
  sentence = tokenizer.decode(output[0].numpy().tolist())
  chatbot_response = sentence.split(' ')[1].replace('', '')
  return chatbot_response

In [ ]:
return_answer_by_chatbot('안녕! 반가워~')